In [1]:
import sys
import os
import pandas as pd
# 현재 작업 디렉토리 기준으로 src 디렉토리 추가
notebook_dir = os.getcwd()  # 현재 Jupyter Notebook의 작업 디렉토리

sys.path.append(os.path.dirname(notebook_dir))


In [2]:
from src.tools.google_drive import ensure_data_files

# 데이터 파일 경로 가져오기
data_paths = ensure_data_files()
data_paths


기존 data가 존재합니다. 파일 경로를 반환합니다.


{'reviewer': '/Users/seongrokkim/Documents/Github/yamyam-lab/data/reviewer.csv',
 'review_keyword': '/Users/seongrokkim/Documents/Github/yamyam-lab/data/review_keyword.csv',
 'category': '/Users/seongrokkim/Documents/Github/yamyam-lab/data/diner_category_raw.csv',
 'review': '/Users/seongrokkim/Documents/Github/yamyam-lab/data/review.csv',
 'dinr_menu': '/Users/seongrokkim/Documents/Github/yamyam-lab/data/dinr_menu.csv',
 'diner': '/Users/seongrokkim/Documents/Github/yamyam-lab/data/diner.csv'}

In [3]:
# load data
diner = pd.read_csv(data_paths["diner"])
diner_category = pd.read_csv(data_paths["category"])

diner = pd.merge(diner, diner_category, on="diner_idx", how="left")


review = pd.read_csv(data_paths["review"])
reviewer = pd.read_csv(data_paths["reviewer"])

review = pd.merge(review, reviewer, on="reviewer_id", how="left")

/var/folders/tn/fxpghkds0jb3fjp_9_k5lwg40000gn/T/ipykernel_17609/2461593576.py:2: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  diner = pd.read_csv(data_paths["diner"])


In [19]:
import pandas as pd
# 날짜 타입으로 변환
review["reviewer_review_date"] = pd.to_datetime(review["reviewer_review_date"])

# 1. 리뷰어별 리뷰 수 및 작성 기간 계산
agg_df = (
    review.groupby("reviewer_id")["reviewer_review_date"]
    .agg(["count", lambda x: (x.max() - x.min()).days])
    .rename(columns={"count": "review_count", "<lambda_0>": "review_period_days"})
    .reset_index()
)

# 2. 짧은 기간에 리뷰 많이 남긴 리뷰어 추출 (예: 5개 이상 리뷰, 3일 이하 기간)
suspicious_reviewers = agg_df[
    (agg_df["review_count"] >= 5) & (agg_df["review_period_days"] <= 3)
]["reviewer_id"]

# 3. 해당 리뷰어들의 리뷰 중 4점 초과만 필터링
high_score_reviews = review[
    (review["reviewer_id"].isin(suspicious_reviewers)) &
    (review["reviewer_review_score"] >= 4) &
    (review["reviewer_avg"] >= 4)
]

# 4. 음식점별 리뷰 수 카운트
diner_counts = high_score_reviews["diner_idx"].value_counts()

# 5. 리뷰 수가 10개 이상인 음식점만 추출
target_diners = diner_counts[diner_counts >= 10].index

# 6. 해당 음식점 리뷰만 보기
suspicious_reviews = high_score_reviews[high_score_reviews["diner_idx"].isin(target_diners)]

# 결과 확인
suspicious_reviews = suspicious_reviews.sort_values(["diner_idx", "reviewer_id", "reviewer_review_date"])
suspicious_reviews.head()


,Unnamed: 0,diner_idx,reviewer_id,review_id,reviewer_review,reviewer_review_date,reviewer_review_score,reviewer_user_name,reviewer_review_cnt,reviewer_avg,badge_grade,badge_level
153811,154111,8025375.0,373970057,650036,(5-추천 3-쏘쏘 1-비추천) 평양냉면의 진가를 맛볼 수 있는 곳입니다. 평양냉면...,2020-05-07,5.0,정상체온,5,4.2,실버 배지,26
198134,198523,8025375.0,386714166,1787115,NaN,2016-07-13,5.0,김민석,15,4.4,브론즈 배지,5
386508,387246,8025375.0,518719358,9878860,NaN,2024-04-07,5.0,박수형,9,4.1,브론즈 배지,7
1318060,1320905,8025375.0,1207185707,6591774,가끔 생각나는맛 하지만 가격은 사악함,2022-11-22,4.0,iliililililliili,8,4.5,브론즈 배지,7
1324475,1327327,8025375.0,1208896970,11322798,NaN,2024-10-30,5.0,Chris,10,4.2,브론즈 배지,6


In [20]:
suspicious_reviews['diner_idx'].value_counts()

diner_idx
4.695770e+08    21
8.453998e+06    19
8.025375e+06    17
1.154752e+07    15
1.716327e+07    13
2.504847e+07    13
1.578746e+09    12
1.886021e+08    11
1.351221e+09    11
1.037514e+07    10
9.093207e+08    10
1.565629e+09    10
1.142427e+09    10
8.761356e+08    10
8.989707e+08    10
8.909929e+08    10
1.033241e+07    10
4.006506e+08    10
2.145579e+07    10
1.788525e+09    10
Name: count, dtype: int64

In [21]:
# 1. 음식점별 날짜별 리뷰 수 집계
date_overlap = (
    suspicious_reviews.groupby(["diner_idx", "reviewer_review_date"])
    .size()
    .reset_index(name="review_count_per_date")
)

# 2. 각 음식점에서 가장 많이 겹친 날짜의 리뷰 수 추출
max_overlap_per_diner = (
    date_overlap.groupby("diner_idx")["review_count_per_date"]
    .max()
    .reset_index(name="max_reviews_on_same_date")
)

# 3. 겹치는 수 기준으로 음식점 정렬
sorted_diners = max_overlap_per_diner.sort_values(by="max_reviews_on_same_date", ascending=False)

# 4. suspicious_reviews에 merge해서, 겹친 수 기준 정렬 정보 부여
suspicious_reviews_with_overlap = suspicious_reviews.merge(sorted_diners, on="diner_idx")

# 5. 최종 정렬된 suspicious_reviews 보기 (많이 겹친 순서)
final_result = suspicious_reviews_with_overlap.sort_values(
    by=["max_reviews_on_same_date", "diner_idx", "reviewer_review_date"],
    ascending=[False, True, True]
)

final_result

,Unnamed: 0,diner_idx,reviewer_id,review_id,reviewer_review,reviewer_review_date,reviewer_review_score,reviewer_user_name,reviewer_review_cnt,reviewer_avg,badge_grade,badge_level,max_reviews_on_same_date
129,759621,4.695770e+08,787775066,4800903,중독성있는 라멘스프\n매우 특이한 분위기와 다소 불편한 좌석\n이베리코를 썼다는 차...,2021-12-28,4.0,수린,9,4.2,실버 배지,19,5
134,1740620,4.695770e+08,1539807943,4925846,먹어본 라멘 중에 제일 맛있어요!!!\n생각보다 유자가 라멘이랑 엄청 잘 어울려요!...,2022-01-21,5.0,.,6,4.1,실버 배지,18,5
130,801144,4.695770e+08,815711074,6055029,한 5번은 간 것 같은데 항상 대만족합니다\n맛있고 직원분들도 친절하셔요\n사리추가...,2022-08-15,5.0,이뿌갈비,5,4.5,브론즈 배지,10,5
133,1411532,4.695770e+08,1248486832,7208903,웨이팅 없이 딱 맞게 들어갔어서 그냥저냥 맛있게 먹음,2023-03-21,4.0,밥값보이,12,4.1,실버 배지,14,5
144,2227017,4.695770e+08,2960513173,8307993,몇년간 단골이었는데 갈수록 사람이 많아져 먹기 힘드네요. 면 반개 추가하면 딱 좋습니다.,2023-08-13,5.0,ㄷㄷㄷ,5,4.3,브론즈 배지,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,10243,1.578746e+09,256355552,6634035,마늘바게트 진짜 최고ㅠㅠㅠㅠ,2022-12-01,5.0,ㅇㅅㅇ,5,4.3,브론즈 배지,8,1
228,1830785,1.578746e+09,1644586585,6758071,마농바게트 두번사세요 아니 세번사세요!!!,2022-12-27,5.0,퀸루나,8,4.5,실버 배지,11,1
230,2072868,1.578746e+09,2171440465,7529979,NaN,2023-05-04,5.0,이보석,7,4.5,브론즈 배지,3,1
224,1258995,1.578746e+09,1164388609,8437114,살면서 먹은 마늘바게트 중에 제일 맛있었어요.\n적당한 촉촉함과 마늘향기 조화가 미...,2023-08-29,5.0,맛난곳만남김,5,5.0,실버 배지,13,1
